# Prompt Engineering using GPT models and LangChain

This notebook contains useful prompt engineering conepts and techniques for learning purpose. The notebook is based on online materials. 

In [16]:
# Install openai.
!pip install openai==0.28.0 langchain==0.0.293 typing-extensions==4.8.0 pandas==2.0.3

Defaulting to user installation because normal site-packages is not writeable


In [17]:
# Import the os package.
import os

# Import the openai package.
import openai

# Set openai.api_key to the OPENAI_API_KEY environment variable.
openai.api_key = os.getenv("OPENAI_API_KEY")

We import both complete and chat complete models. Completion models include gpt-1, gpt-2, and gpt-3 etc. Chat completion models include gpt-3.5-turbo and gpt-4.

In [18]:
# Import OpenAI.
from langchain.llms import OpenAI

# Import ChatOpenAI.
from langchain.chat_models import ChatOpenAI

## 1. Import the Financial News Headlines Data

A small sample of financial headlines is stored in `financial_headlines.txt`.

Our first step is to read in the text file and store the headlines in a Python list.

In [19]:
# Open the text file and read its lines.
headlines = []
with open("financial_headlines.txt") as f:
    headlines = f.readlines()

headlines = [line.strip("\n") for line in headlines]

# Print all headlines.
print(headlines)

["Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .", 'Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .', 'Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .', 'Tiimari , the Finnish retailer , reported to have geenrated quarterly revenues totalling EUR 1.3 mn in the 4th quarter 2009 , up from EUR 0.3 mn loss in 2008 .', "Finnish Metso Paper has been awarded a contract for the rebuild of Sabah Forest Industries ' ( SFI ) pulp mill in Sabah , Malaysia .", 'Finnish Outokumpu Technology has been awarded several new grinding technology contracts .']


## 2: Setting up Prompt Templates

Prompt templates help us to organize our code to be modular, scalable and reusable by its following properties:
* allows for dynamic prompts
* built-in verification tools 
* easily saved, versioned and integrated into the code base

We will set up Prompt Templates from the `langchain` package to automatically determine financial sentiment from the headlines and extract relevant company names. 

### 2.1 PromptTemplate for completion models
* Prompt templates allow us to insert the values dynamically
* As demonstrated here, we can change the values of 'headline' and build different prompts for different use cases

In [20]:
# Import the PromptTemplate class.
from langchain.prompts import PromptTemplate

# Create a dynamic template to analyze a single headline.
prompt_template = PromptTemplate.from_template("Analyze the following financial headline for sentiment: {headline}")

# Format the prompt template on the first headline of the dataset.
formatted_prompt = prompt_template.format(headline=headlines[0])

# Print the formatted template.
print(formatted_prompt)

Analyze the following financial headline for sentiment: Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .


### 2.2 ChatPromptTemplate

a `ChatPromptTemplate`are used to communicate with conversational models like GPT-4 and GPT-3.5-Turbo. One advantage of this type of the template is that we can define a system message.

In terms of prompt engineering, system messages can help to improve the quality of the output by the following ways:
- Define a role: *"You are a X", "Your role is to do X", ...*
- Define a tone of voice: *"Respond in a formal manner", "Use customer-oriented language", ...*
- Define restrictions on output format: *"The format of the output is X", "The output is strictly limited to X, Y, Z", ...*
- Define a scope: *"Only answer questions on topic X", "If the user questions is not about X, answer with Y", ...*

In [21]:
# Import the ChatPromptTemplate class.
from langchain.prompts import ChatPromptTemplate

# Define the system message and define the role of the chat model.
system_message = """You are performing sentiment analysis on news headlines regarding financial analysis. 
This sentiment is to be used to advice financial analysts. 
The format of the output has to be consistent. 
The output is strictly limited to any of the following options: [positive, negative, neutral]."""

# Initialize a new ChatPromptTemplate with a system message and human message.
chat_template = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('human', "Analyze the following financial headline for sentiment: {headline}")
])

# Format the ChatPromptTemplate.
formatted_chat_template = chat_template.format(headline=headlines[0])

# Print the formatted template.
print(formatted_chat_template)

System: You are performing sentiment analysis on news headlines regarding financial analysis. 
This sentiment is to be used to advice financial analysts. 
The format of the output has to be consistent. 
The output is strictly limited to any of the following options: [positive, negative, neutral].
Human: Analyze the following financial headline for sentiment: Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .


## 3: Setting up LLM Chains

LLM Chains allow us to combine a model with a prompt template. These chains can be created for both *completion models* and *chat completion models*.

LLM Chains can be used to "chain" prompt flows, by using the output of a previous chain as input for the next.

Let's set up a chain for a completion model first, using the templates that we've just built.

In [22]:
# Import the LLMChain class.
from langchain.chains import LLMChain

# Create the LLMChain by combining a completion model and a prompt.
completion_chain = LLMChain(llm=OpenAI(), prompt=prompt_template)

# Run the LLMChain.
completion_chain.run(headline=headlines[0])

'\n\nPositive'

This looks the same as calling a sentimental analysis by open source llm models using hugging face transformers' pipeline, as demonstrated in one of my notebooks, "using_Huggingface_llms" notebook, which is also in this repo. The difference here is that we used prompt templates.

We can execute on a chat completion model using LLMChain and a chat template

In [23]:
# Create the LLMChain by combining a chat completion model and a prompt.
chat_chain = LLMChain(llm=ChatOpenAI(), prompt=chat_template)

# Run the LLMChain.
chat_chain.run(headline=headlines[0], system_message=system_message)

'The sentiment of the given financial headline is positive.'

## 4: Working with Agents and Tools

### What are Tools and Agents?

We can access (external) tools using the output of the GPT-model. Large language models output only text. In order to call a function (to access a tool) based on the text output of a large language model, we can use agents. They can parse the text output, pick the correct tool and define its input.

In this example, we will make use of the `PythonREPLTool`. This allows the GPT-model to run the Python code that it generates, and can be useful for carrying out tasks, such as complicated calculations, which is not what large language models were trained for.

Always set the `max_tokens` argument to prevent endless recursive calls!

In [24]:
# Import the necessary classes from langchain.
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

# Instantiate a Python agent, with the PythonREPLTool.
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

# Ask the agent for the solution of a mathematical equation.
agent_executor.run("What is the square root of 250? Round the answer down to 4 decimals.")



> Entering new AgentExecutor chain...
 I need to calculate the square root of 250
Action: Python_REPL
Action Input: import math; print(round(math.sqrt(250), 4))
Observation: 15.8114

Thought: I now know the final answer
Final Answer: 15.8114

> Finished chain.


'15.8114'

The agent correctly identified PythonREPLTool for the calculation. Now let's demonstrate a more complex task: writing the python code to generate a pandas datafram that contains company names, their sentimental analysis results, and the original headlines. Finally, save the dataframe as a csv file with the file name specified in the prompt.

In [25]:
# Run the agent
agent_executor.run(f"""For every of the following headlines, extract the company name and whether the financial sentiment is positive, neutral or negative. 
Load this data into a pandas dataframe. 
The dataframe will have three columns: the name of the company, whether the financial sentiment is positive or negative and the headline itself. 
The dataframe can then be saved in the current working directory under the name financial_analysis.csv.
If a csv file already exists with the same name, it should be overwritten.

The headlines are the following:
{headlines}""")



> Entering new AgentExecutor chain...
 I need to extract the company name and sentiment from each headline
Action: Python_REPL
Action Input: 
import pandas as pd

headlines = ["Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .", 'Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .', 'Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .', 'Tiimari , the Finnish retailer , reported to have geenrated quarterly revenues totalling EUR 1.3 mn in the 4th quarter 2009 , up from EUR 0.3 mn loss in 2008 .', "Finnish Metso Paper has been awarded a contract for the rebuild of Sabah Forest Industries ' ( SFI ) pulp mill in Sabah , Malaysia .", 'Finnish Outok

'The dataframe has been saved in the current working directory under the name financial_analysis.csv. It contains three columns: the name of the company, whether the financial sentiment is positive or negative and the headline itself.'

Now, let' check if csv file contains the information about company names, the sentimental analysis results and original headline content.

In [26]:
import pandas as pd
data = pd.read_csv("financial_analysis.csv")
data

,company_name,sentiment,headline
0,Aktia,positive,Finnish Aktia Group 's operating profit rose t...
1,measuring,negative,Finnish measuring equipment maker Vaisala Oyj ...
2,pharmaceuticals,positive,Finnish pharmaceuticals company Orion reports ...
3,",",negative,"Tiimari , the Finnish retailer , reported to h..."
4,Metso,positive,Finnish Metso Paper has been awarded a contrac...
5,Outokumpu,positive,Finnish Outokumpu Technology has been awarded ...


## 5: Adding Few Shot Learning

Few shot learning means adding examples into our prompt to guide the model for more relavant responses. 

There are three categories of N shot learning:
- Few shot leaning (multiple examples)
- Single shot learning (one example)
- Zero shot leaning (no examples)

Few shot learning generally gives better results, as more information is provided about our desired results.

The next section showed the following techniques
* how to build the prompts with examples in few shot learing. It is pretty straightforward: you just embed the examples in the prompt templates and run the model!
* how to format the output as a comma separated list using langchain's output parsers

In [30]:
# Import the CommaSeparatedListOutputParser class.
from langchain.output_parsers import CommaSeparatedListOutputParser

# Instantiate the output parser.
output_parser = CommaSeparatedListOutputParser()

# Get the format instructions from the output parser.
format_instructions = output_parser.get_format_instructions()

# Store the few shot examples in a variable.
sentiment_examples = """
If a company is doing financially better than before, the sentiment is positive. For example, when profits or revenue have increased since the last quarter or year, exceeding expectations, a contract is awarded or an acquisition is announced.
If the company's profits are decreasing, losses are mounting up or overall performance is not meeting expectations, the sentiment is negative.
If nothing positive or negative is mentioned from a financial perspective, the sentiment is neutral.
"""

# Instantiate a new prompt template with the format instructions.
sentiment_template = PromptTemplate(
    template="Get the financial sentiment of each of the following headlines. {few_shot_examples} The output is strictly limited to any of the following options: ['Positive', 'Negative', 'Neutral']: {headlines}.\n{format_instructions}",
    input_variables =["headlines", "few_shot_examples"],
    partial_variables={"format_instructions": format_instructions}    
)

# Format the template.
formatted_sentiment_template = sentiment_template.format(headlines=headlines, few_shot_examples=sentiment_examples)

# Run the model on the formatted template.
model = OpenAI(temperature=0)
_output = model(formatted_sentiment_template)

# Parse the model output.
sentiments = output_parser.parse(_output)

print(sentiments)

['Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive']


## Summary

This notebook demonstates several commonly used prompt engineering techniques, including
- Using SystemMessage to optimize the outcomes
- Setting up prompt templates 
- Using LLMChains
- Using LangChain Agents and Tools to add additional functionalities inculding complex calculations and writing code, as demonstrated in this notebook, to generative AI projects
- How to build prompts for few shot learning